In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
from requests_html import HTMLSession, AsyncHTMLSession

In [3]:
rejected = [
    'wa.link',
    'mailto:',
    't.me',
    'tally.so',
    'discord.gg',
    'arxiv.org',
    'javascript:'
]

In [4]:
async def get_html(url):
    session = AsyncHTMLSession(verify = False)
    result = None
    try:
        
        r = await session.get(url, verify = False)
        await r.html.arender()
        
        result = r.html.html
    except Exception as e:
        pass
    
    await session.close()
    return result


def get_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    links = []
    for link in soup.find_all('a'):
        links.append(link.get('href'))
    return links

In [5]:
import asyncio

visited_pages = set(['https://www.kln.gov.my/'])

async def crawl(base_url):
    if base_url.endswith('.pdf'):
        return
    
    html = await get_html(base_url)
    
    if html is None:
        return

    visited_pages.add(base_url)
    links = get_links(html)
    for link in links:
        try:
            if any([r in link for r in rejected]):
                continue

            if not link.startswith('http'):
                link = urljoin(base_url, link)

            link = link.split('#')[0]
            link = link.split('?')[0]

            if link not in visited_pages:
                visited_pages.add(link)
        except Exception as e:
            pass

In [6]:
from tqdm import tqdm

batch_size = 10

for no in range(3):
    l = list(visited_pages)
    print(no, len(l))
    for i in tqdm(range(0, len(l), batch_size)):
        b = l[i: i + batch_size]
        await asyncio.gather(*[crawl(url) for url in b])

0 1


100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.23s/it]


1 91


 10%|████▍                                       | 1/10 [00:05<00:50,  5.64s/it]Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error Target.detachFromTarget: Target closed.')>
pyppeteer.errors.NetworkError: Protocol error Target.detachFromTarget: Target closed.
Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id
100%|███████████████████████████████████████████| 10/10 [01:04<00:00,  6.46s/it]


2 1506


 13%|█████▎                                    | 19/151 [06:26<36:10, 16.44s/it]Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error Target.detachFromTarget: Target closed.')>
pyppeteer.errors.NetworkError: Protocol error Target.detachFromTarget: Target closed.
Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id
 13%|█████▌                                    | 20/151 [06:32<29:15, 13.40s/it]Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error Target.sendMessageToTarget: Target closed.')>
pyppeteer.errors.NetworkError: Protocol error Target.sendMessageToTarget: Target closed.
Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendM

In [7]:
len(visited_pages)

10585

In [8]:
visited_pages

{'https://www.spr.gov.my/aduan-pertanyaan',
 'https://www.data.gov.my/data/en_US/dataset/list-of-asean-som-meetings-2018',
 'https://www.searcct.gov.my/wp-content/uploads/2020/09/Searcct-Papercutting-June-3.jpg',
 'https://www.facebook.com/help/146952742043748/Bolehkah+saya+menyekat+atau+menyembunyikan+iklan+yang+dipaparkan+pada+akaun+Facebook+saya%3F/',
 'https://www.idfr.gov.my/archived-articles/1021-new-director-general-of-idfr-sp-12667',
 'https://www.instagram.com/explore/locations/',
 'https://www.imi.gov.my/index.php/en/main-services/malaysia-my-second-home-mmh2-en/',
 'https://www.eperolehan.gov.my/documents/10182/100000025911136/kelantan_2019_%28128%29.pdf/84ff841d-e88e-4336-b859-ea6fbc6983ee',
 'https://mosti.spab.gov.my/eApps/system/index.do',
 'http://www.opcw.org/media-centre/featured-topics/ukraine',
 'https://www.diplomatie.gouv.fr/fr/dossiers-pays/malaisie/evenements/article/malaisie-entretien-de-catherine-colonna-avec-son-homologue-02-06-23',
 'https://microsoft.com/ed

In [9]:
import json

with open('url-kln.gov.my.json', 'w') as fopen:
    json.dump(list(visited_pages), fopen)